In [1]:
import os
import json
import shutil
import random
from pathlib import Path

In [2]:
def split_dataset(data_path, output_path, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1, random_seed=42):
    """
    Divide el dataset de INE en conjuntos de train, val y test.
    
    Args:
        data_path (str): Ruta a la carpeta con las imágenes y metadata.jsonl
        output_path (str): Ruta donde se creará la carpeta 'sets'
        train_ratio (float): Porcentaje para entrenamiento (default: 0.7)
        val_ratio (float): Porcentaje para validación (default: 0.2)
        test_ratio (float): Porcentaje para prueba (default: 0.1)
        random_seed (int): Semilla para reproducibilidad (default: 42)
    """
    
    # Validar que los porcentajes sumen 1.0
    if abs(train_ratio + val_ratio + test_ratio - 1.0) > 0.001:
        raise ValueError("Los porcentajes de train, val y test deben sumar 1.0")
    
    # Configurar semilla para reproducibilidad
    random.seed(random_seed)
    
    # Rutas
    data_path = Path(data_path)
    metadata_file = data_path / "metadata.jsonl"
    output_path = Path(output_path) / "sets"
    
    # Verificar que existan los archivos necesarios
    if not data_path.exists():
        raise FileNotFoundError(f"No se encontró la carpeta: {data_path}")
    if not metadata_file.exists():
        raise FileNotFoundError(f"No se encontró el archivo: {metadata_file}")
    
    print("🔄 Iniciando división del dataset...")
    print(f"📁 Carpeta de datos: {data_path}")
    print(f"📄 Archivo metadata: {metadata_file}")
    print(f"📤 Carpeta de salida: {output_path}")
    print(f"📊 Distribución: Train {train_ratio*100}% | Val {val_ratio*100}% | Test {test_ratio*100}%")
    print("-" * 70)
    
    # Leer metadata
    metadata_entries = []
    with open(metadata_file, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                entry = json.loads(line)
                metadata_entries.append(entry)
    
    print(f"📊 Total de entradas en metadata: {len(metadata_entries)}")
    
    # Obtener lista de imágenes que existen físicamente
    image_files = []
    for entry in metadata_entries:
        image_path = data_path / entry['file_name']
        if image_path.exists():
            image_files.append(entry)
        else:
            print(f"⚠️  Advertencia: No se encontró la imagen {entry['file_name']}")
    
    print(f"📸 Imágenes encontradas: {len(image_files)}")
    
    # Mezclar aleatoriamente
    random.shuffle(image_files)
    
    # Calcular tamaños de cada conjunto
    total_images = len(image_files)
    train_size = int(total_images * train_ratio)
    val_size = int(total_images * val_ratio)
    test_size = total_images - train_size - val_size  # El resto va a test
    
    print(f"🔢 Distribución final:")
    print(f"   📚 Train: {train_size} imágenes")
    print(f"   🔍 Val: {val_size} imágenes")
    print(f"   🧪 Test: {test_size} imágenes")
    print("-" * 70)
    
    # Dividir los datos
    train_data = image_files[:train_size]
    val_data = image_files[train_size:train_size + val_size]
    test_data = image_files[train_size + val_size:]
    
    # Crear carpetas
    sets = {
        'train': (train_data, output_path / 'train'),
        'val': (val_data, output_path / 'val'),
        'test': (test_data, output_path / 'test')
    }
    
    for set_name, (data, set_path) in sets.items():
        print(f"📂 Creando conjunto: {set_name}")
        
        # Crear carpeta si no existe
        set_path.mkdir(parents=True, exist_ok=True)
        
        # Ordenar los datos por file_name para mantener orden consistente
        data_sorted = sorted(data, key=lambda x: x['file_name'])
        
        # Copiar imágenes
        print(f"   📋 Copiando {len(data_sorted)} imágenes...")
        for entry in data_sorted:
            src_image = data_path / entry['file_name']
            dst_image = set_path / entry['file_name']
            shutil.copy2(src_image, dst_image)
        
        # Crear metadata.jsonl
        print(f"   📝 Creando metadata.jsonl...")
        metadata_path = set_path / 'metadata.jsonl'
        with open(metadata_path, 'w', encoding='utf-8') as f:
            for entry in data_sorted:
                json_line = json.dumps(entry, ensure_ascii=False, separators=(', ', ': '))
                f.write(json_line + '\n')
        
        print(f"   ✅ {set_name} completado: {len(data_sorted)} archivos")
    
    print("-" * 70)
    print("🎉 ¡División del dataset completada exitosamente!")
    print(f"📁 Los conjuntos se guardaron en: {output_path}")
    
    # Mostrar estructura final
    print("\n📋 Estructura creada:")
    for set_name in ['train', 'val', 'test']:
        set_path = output_path / set_name
        if set_path.exists():
            image_count = len([f for f in set_path.iterdir() if f.suffix.lower() in ['.jpg', '.jpeg', '.png']])
            print(f"   📂 {set_name}/")
            print(f"      📸 {image_count} imágenes")
            print(f"      📄 metadata.jsonl")

In [3]:
def main():
    """Función principal para ejecutar la división del dataset"""
    
    # Configuración de rutas
    data_path = r"C:\Users\Adrian\Desktop\reyi\inesdataset_final\data"
    output_path = r"C:\Users\Adrian\Desktop\reyi\inesdataset_final"
    
    # Configuración de splits (puedes modificar estos valores)
    train_ratio = 0.7  # 70% para entrenamiento
    val_ratio = 0.2    # 20% para validación
    test_ratio = 0.1   # 10% para prueba
    
    try:
        split_dataset(
            data_path=data_path,
            output_path=output_path,
            train_ratio=train_ratio,
            val_ratio=val_ratio,
            test_ratio=test_ratio,
            random_seed=42  # Para reproducibilidad
        )
    except Exception as e:
        print(f"❌ Error: {e}")

In [4]:
if __name__ == "__main__":
    main()

🔄 Iniciando división del dataset...
📁 Carpeta de datos: C:\Users\Adrian\Desktop\reyi\inesdataset_final\data
📄 Archivo metadata: C:\Users\Adrian\Desktop\reyi\inesdataset_final\data\metadata.jsonl
📤 Carpeta de salida: C:\Users\Adrian\Desktop\reyi\inesdataset_final\sets
📊 Distribución: Train 70.0% | Val 20.0% | Test 10.0%
----------------------------------------------------------------------
📊 Total de entradas en metadata: 30
⚠️  Advertencia: No se encontró la imagen ine_falsa_271.jpg
⚠️  Advertencia: No se encontró la imagen ine_falsa_272.jpg
⚠️  Advertencia: No se encontró la imagen ine_falsa_273.jpg
⚠️  Advertencia: No se encontró la imagen ine_falsa_274.jpg
⚠️  Advertencia: No se encontró la imagen ine_falsa_275.jpg
⚠️  Advertencia: No se encontró la imagen ine_falsa_276.jpg
⚠️  Advertencia: No se encontró la imagen ine_falsa_277.jpg
⚠️  Advertencia: No se encontró la imagen ine_falsa_278.jpg
⚠️  Advertencia: No se encontró la imagen ine_falsa_279.jpg
⚠️  Advertencia: No se encontró 